In [1]:
import pandas as pd
from pandas_datareader import data
import numpy as np
from datetime import datetime
from datetime import timedelta
import re
import math
import statsmodels.api as sm
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [2]:
with open('tickers.txt','r') as file:
    tickers = file.readlines()
    
tickers = [item.strip() for item in tickers]

In [3]:
columns = ['transaction_date', 'transaction_type', 'company_name',
           'ticker', 'name', 'position', 'shares_traded', 'avg_price',
           'total_price', 'shares_owned', 'own_type']



In [4]:
def file_readr(ticker_list):
    my_dict = {}
    for ticker in ticker_list:
        try:
            current_df = pd.read_csv(f'./insiders_{ticker}.csv', header = None, names = columns)
            if current_df.shape[0] > 0:
                my_dict[ticker] = current_df
            else:
                continue
            del current_df
        except:
            continue
    
    return my_dict



In [104]:
dict_insiders = file_readr(tickers)




### cleaning insiders dictionary 

In [105]:

def clean_dict(x_dict):
    
    for ticker in x_dict.keys():
        df = x_dict[ticker]
    
        df.set_index(pd.to_datetime(df.transaction_date), inplace = True)

        df.drop('transaction_date',axis=1, inplace = True)
    
        transactions_list = [item.split('(')[0] for item in df.transaction_type.tolist()]
    
        df['sale_num'] = [1 if item == 'Sale' else 0 for item in transactions_list]
        df['buy_num'] = [1 if item == 'Purchase' else 0 for item in transactions_list]

        weekly = df.set_index(df.index-timedelta(days=7)).resample('W-SUN').sum()[['shares_traded','total_price','sale_num','buy_num']]
    
        weekly.index.name = 'Date'
    
        x_dict[ticker] = weekly
        
        print(f'{ticker} cleaned successfully')
    
    return x_dict
    

In [106]:
dict_insiders = clean_dict(dict_insiders)

FCX cleaned successfully
MAS cleaned successfully
HAS cleaned successfully
AMZN cleaned successfully
MU cleaned successfully
VAR cleaned successfully
PEG cleaned successfully
LEG cleaned successfully
JBHT cleaned successfully
CAT cleaned successfully
K cleaned successfully
HES cleaned successfully
NDAQ cleaned successfully
BLL cleaned successfully
AAP cleaned successfully
VTR cleaned successfully
LW cleaned successfully
PG cleaned successfully
NSC cleaned successfully
PBCT cleaned successfully
AMAT cleaned successfully
MAR cleaned successfully
INCY cleaned successfully
ORLY cleaned successfully
ROL cleaned successfully
JWN cleaned successfully
NEE cleaned successfully
QRVO cleaned successfully
CVX cleaned successfully
HUM cleaned successfully
NBL cleaned successfully
BDX cleaned successfully
AWK cleaned successfully
SJM cleaned successfully
PXD cleaned successfully
CHD cleaned successfully
AMP cleaned successfully
DRI cleaned successfully
AEE cleaned successfully
ABMD cleaned successfu

### cleaning stock price data

In [27]:
def get_stocks(x_dict):
    stocks_dict = {}
    
    for ticker in x_dict.keys():
        start_date = str(x_dict[ticker].index[0]).split()[0]
        end_date = str(x_dict[ticker].index[-1]).split()[0]
        
        try:
            stocks_df = data.DataReader(ticker,'yahoo',start_date,end_date)
        except:
            continue
        
        stocks_dict[ticker]=stocks_df
        
    return stocks_dict
        
        

In [31]:
def clean_stocks(x_dict):    
    stocks_dict = {}
    
    for item in x_dict.keys():
        ticker = item
        stocks_df = x_dict[ticker]
        stocks_df['Return'] = stocks_df['Adj Close'].diff()
        
        stocks_weekly = stocks_df.set_index(stocks_df.index-timedelta(days=7)).resample('W-SUN').std()[['Return']]
        
        stocks_dict[ticker]=stocks_weekly
        
    return stocks_dict

In [62]:
stocks_dict_pre = get_stocks(dict_insiders)

In [64]:
stocks_dict = clean_stocks(stocks_dict_pre)

False

#### merging and cleaning insider & stock data

In [159]:
def merge_dicts(x,y):
    full_dict = {}
    x_dict = x
    y_dict = y
    
    
    for ticker in x_dict.keys():
        
        if ticker not in y_dict.keys():
            continue
            
        else:
            
            index = [item for item in list(zip(x_dict[ticker].index,y_dict[ticker].index))]
            index_test = all([item[0] == item[1] for item in index])
            
            if index_test:
                index = [item[0] for item in index]
                
                try:
                    
                    left = x_dict[ticker].loc[index]
                    right = y_dict[ticker].loc[index]

                    full_df = pd.merge(left,right, on=left.index)

                    full_df = full_df.set_index('key_0')

                    full_df.index.name = 'Date'

                    full_df.sale_num[full_df.sale_num > 0] = 1

                    full_df.buy_num[full_df.buy_num>0] = 1

                    full_df['risk_dummy'] = np.where(full_df.Return > 1.5, 1, 0)

                    full_dict[ticker]=full_df

                except:
                    print(f'error on ticker:{ticker}')
            
    return full_dict
        
        
        

In [160]:
full_dict = merge_dicts(dict_insiders,stocks_dict)

/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
### plotting insider trades

In [ ]:

def plot_dict(insider_dict,stocks_dict):
    plt.style.use('bmh')
    fig = plt.figure(figsize=(20,15))
    
    for item in insider_dict.keys():
        
        ticker = item
        
        if ticker not in stocks_dict.keys():
            
            raise ValueError('Keys must match in both dictionaries')
            
        else:
            
            stock_df = stocks_dict[ticker]
            insider_df = insider_dict[ticker]
            
            sns.lineplot(x=stock_df.index,y=stock_df['Adj Close'])
            for x in insider_df.index.tolist():
                plt.axvline(x,linestyle=':',linewidth=0.1,c='b')
                    
        

In [ ]:
plot_dict(dict_insiders,stocks_dict_pre)

### fitting logistic models

In [ ]:
def logit_dict(x_dict):
    fit_dict = {}
    
    for item in x_dict.keys():
        
        ticker = item
        print(ticker)
        df = x_dict[ticker]
        
        X = df[['shares_traded','sale_num','buy_num']]
        Y = df.return_dummy
        logit_model = sm.Logit(Y,X)
        result = logit_model.fit()
        
        fit_dict[ticker]=result
        
        
    return fit_dict

In [ ]:
sm.Logit().get

In [ ]:
models_dict = logit_dict(full_dict)



In [ ]:
for key,value in models_dict.items():
    print(key)
    print('-'*30)
    print(value.pvalues)
    print('*'*30)


